# INTRODUCTION
Le but de ce notebook est de relever les anomalies présentes sur les notices d'exemplaire afin de pouvoir exporter des listes à destination des aquéreurs. Le script se compose des parties suivantes :
1. Import de la table `items`
2.Traduction des colonnes pour les rendre lisibles pour les collègues qui auront le tableau sous les yeux 
3. Import des tables de valeurs autorisées
4. Une suite de requêtes sur concernant les différentes colonnes de notre dataframe nommé `items`

# IMPORT DE LA TABLE `items`

In [1]:
import pandas as pd
from datetime import datetime

from kiblib.utils.db import DbConn

In [2]:
db_conn = DbConn().create_engine()

On définit la variable query comme une requête SQL dans laquelle aux champs de la table items on a ajouté le champs itemtype de la table biblioitems:

In [3]:
query = """SELECT i.itemnumber, i.biblionumber, i.biblioitemnumber, i.barcode, i.dateaccessioned, i.booksellerid, i.homebranch, i.price, i.replacementprice, i.replacementpricedate, i.datelastborrowed, i.datelastseen, i.stack, i.notforloan, i.damaged, i.damaged_on, i.itemlost, i.itemlost_on, i.withdrawn, i.withdrawn_on, i.itemcallnumber, i.coded_location_qualifier, i.issues, i.renewals, i.reserves, i.restricted, i.itemnotes, i.itemnotes_nonpublic, i.holdingbranch,i.timestamp, i.location, i.permanent_location, i.onloan, i.cn_source, i.cn_sort, i.ccode, i.materials, i.uri, i.itype, i.more_subfields_xml, i.enumchron, i.copynumber, i.stocknumber, i.new_status, i.exclude_from_local_holds_priority, bi.itemtype
FROM koha_prod.items i
JOIN koha_prod.biblioitems bi ON bi.biblionumber = i.biblionumber """

In [4]:
items = pd.read_sql(query,db_conn)

# MODIFICATION DU DATAFRAME & AMÉLIORATION DE LA LISIBILITÉ

In [5]:
items.columns # avant la traduction

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'barcode',
       'dateaccessioned', 'booksellerid', 'homebranch', 'price',
       'replacementprice', 'replacementpricedate', 'datelastborrowed',
       'datelastseen', 'stack', 'notforloan', 'damaged', 'damaged_on',
       'itemlost', 'itemlost_on', 'withdrawn', 'withdrawn_on',
       'itemcallnumber', 'coded_location_qualifier', 'issues', 'renewals',
       'reserves', 'restricted', 'itemnotes', 'itemnotes_nonpublic',
       'holdingbranch', 'timestamp', 'location', 'permanent_location',
       'onloan', 'cn_source', 'cn_sort', 'ccode', 'materials', 'uri', 'itype',
       'more_subfields_xml', 'enumchron', 'copynumber', 'stocknumber',
       'new_status', 'exclude_from_local_holds_priority', 'itemtype'],
      dtype='object')

In [6]:
items = items.rename(columns={"barcode":"code-barre",
                              "dateaccessioned":"date d'acquisition",
                              "homebranch":"site propriétaire",
                              "holdingbranch":"site dépositaire",
                              "price":"prix",
                              "replacementprice":"coût de remplacement",
                              "datelastborrowed":"date dernier emprunt",
                              "datelastseen":"vu la dernière fois",
                              "damaged_on":"abimé le",
                              "itemlost_on":"perdu le",
                              "withdrawn_on":"retiré de la circulation le",
                              "itemcallnumber":"cote",
                              "issues":"nombre de prêts",
                              "timestamp":"date dernier changement",
                              "permanent_location":"localisation permanente",
                              "onloan":"retour prévu le",
                              "itemnotes":"notes publiques",
                              "itemnotes_nonpublic":"notes internes"})

In [7]:
items.columns #après la traduction

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'code-barre',
       'date d'acquisition', 'booksellerid', 'site propriétaire', 'prix',
       'coût de remplacement', 'replacementpricedate', 'date dernier emprunt',
       'vu la dernière fois', 'stack', 'notforloan', 'damaged', 'abimé le',
       'itemlost', 'perdu le', 'withdrawn', 'retiré de la circulation le',
       'cote', 'coded_location_qualifier', 'nombre de prêts', 'renewals',
       'reserves', 'restricted', 'notes publiques', 'notes internes',
       'site dépositaire', 'date dernier changement', 'location',
       'localisation permanente', 'retour prévu le', 'cn_source', 'cn_sort',
       'ccode', 'materials', 'uri', 'itype', 'more_subfields_xml', 'enumchron',
       'copynumber', 'stocknumber', 'new_status',
       'exclude_from_local_holds_priority', 'itemtype'],
      dtype='object')

# Utiliser la jointure de tables pour remplacer les valeurs autorisées

## Notforloan

In [8]:
query2 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'etat'"""

In [9]:
va_notforloan = pd.read_sql(query2, db_conn)

In [10]:
va_notforloan['authorised_value'] = va_notforloan['authorised_value'].astype(int)

In [11]:
items = items.merge(va_notforloan,left_on="notforloan",right_on="authorised_value",how="left")

In [12]:
items = items.rename(columns={"authorised_value": "valeurs autorisées", "lib": "statut"})

In [13]:
items = items.drop(columns=['valeurs autorisées'])

## Damaged

In [14]:
query3 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'damaged'"""

In [15]:
va_damaged = pd.read_sql(query3, db_conn)

In [16]:
va_damaged['authorised_value'] = va_damaged['authorised_value'].astype(int)

In [17]:
items = items.merge(va_damaged,left_on="damaged",right_on="authorised_value",how="left")

In [18]:
items = items.rename(columns={"lib": "abimé"})

In [19]:
items = items.drop(columns=["authorised_value"])

## itemlost

In [20]:
query4 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'lost'"""

In [21]:
va_itemlost = pd.read_sql(query4, db_conn)

In [22]:
va_itemlost['authorised_value'] = va_itemlost['authorised_value'].astype(int)

In [23]:
items = items.merge(va_itemlost,left_on="itemlost",right_on="authorised_value",how="left")

In [24]:
items = items.rename(columns={"lib": "perdu"})

In [25]:
items = items.drop(columns=["authorised_value"])

## Withdrawn

In [26]:
query5 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'retirecoll'"""

In [27]:
va_withdrawn = pd.read_sql(query5, db_conn)

In [28]:
va_withdrawn['authorised_value'] = va_withdrawn['authorised_value'].astype(int)

In [29]:
items = items.merge(va_withdrawn,left_on="withdrawn",right_on="authorised_value",how="left")

In [30]:
items = items.rename(columns={"lib": "retiré de la circulation"})

In [31]:
items = items.drop(columns=["authorised_value"])

## Ccode

In [32]:
query6 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'collection'"""

In [33]:
va_collection = pd.read_sql(query6, db_conn)

In [34]:
items = items.merge(va_collection,left_on="ccode",right_on="authorised_value",how="left")

In [35]:
items = items.rename(columns={"lib": "collection"})

In [36]:
items = items.drop(columns=["authorised_value"])

## location

In [37]:
query7 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'loc'"""

In [38]:
va_location = pd.read_sql(query7, db_conn)

In [39]:
items = items.merge(va_location,left_on="location",right_on="authorised_value",how="left")

In [40]:
items = items.rename(columns={"lib": "localisation"})

In [41]:
items = items.drop(columns=["authorised_value"])

## itemtype

In [42]:
query8 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'typedoc'"""

In [43]:
va_itemtype = pd.read_sql(query8, db_conn)

In [44]:
items = items.merge(va_itemtype,left_on="itemtype",right_on="authorised_value",how="left")

In [45]:
items = items.rename(columns={"lib": "type de support"})

In [46]:
items = items.drop(columns=["authorised_value"])

items.columns

* [x] notforloan / statut 
* [x] damaged / abimé
* [x] itemlost / perdu
* [x] withdrawn / retiré de la circulation
* [x] ccode / code collection
* [x] location / localisation


# DÉFINITION DE QUELQUES VARIABLES IMPORTANTES

## Les colonnes à exporter

In [47]:
colonnes_a_exporter = ["code-barre",
                       "date d'acquisition",
                       "site propriétaire",
                       "site dépositaire",
                       "prix",
                       "coût de remplacement",
                       "date dernier emprunt",
                       "vu la dernière fois",
                       "statut",
                       "abimé",
                       "abimé le",
                       "perdu",
                       "perdu le",
                       "retiré de la circulation",
                       "type de support",
                       "collection",
                       "cote",
                       "nombre de prêts",
                       "date dernier changement",
                       "localisation",
                       "localisation permanente",
                       "notes publiques",
                       "notes internes"
                      ]

## Transformation des colonnes au format `date`

In [48]:
items["date d'acquisition"] = pd.to_datetime(items["date d'acquisition"])

In [49]:
items["date dernier emprunt"] = pd.to_datetime(items["date dernier emprunt"])

In [50]:
items["vu la dernière fois"] = pd.to_datetime(items["vu la dernière fois"])

In [51]:
items["abimé le"] = pd.to_datetime(items["abimé le"])

In [52]:
items["perdu le"] = pd.to_datetime(items["perdu le"])

In [53]:
items["date dernier changement"] = pd.to_datetime(items["date dernier changement"])

## Les deadlines

In [54]:
deadline_6_mois = pd.to_datetime("today") - pd.Timedelta(365/2, unit='D')
deadline_6_mois

Timestamp('2022-06-14 03:08:08.115045')

In [55]:
deadline_1_an = pd.to_datetime("today") - pd.Timedelta(365, unit='D')
deadline_1_an

Timestamp('2021-12-13 15:08:08.148761')

In [56]:
deadline_2_ans = pd.to_datetime("today") - pd.Timedelta(365*2, unit='D')

# ANALYSE DES ANOMALIES
## Analyse sur les données incohérentes

### Site propriétaire vs localisation interne
Il existe 3 sites propriétaires pour la médiathèque de Roubaix : `MED`,`BUS` et `MUS`. Voici une traduction des requêtes effectuées ci-dessous : 
* `anomalies_loc_incoherente_med` : documents dont le site propriétaire est MED et dont le code de localisation ne commence pas par MED
* `anomalies_loc_incoherente_bus` : documents dont le site propriétaire est BUS et donc le code de localisation est différent de BUS.

In [57]:
items.groupby("localisation").size()

localisation
1er étage                           75240
2e étage -  Jeunesse                38106
2e étage - Salle d'étude             1953
3e étage                            19599
Espace Périodiques                      3
Mag Bibliothèque professionnelle     2120
Mag Patrimoine                       4420
Mag Patrimoine BR 4                  3574
Mag Patrimoine BR 8                  3383
Mag Patrimoine Plan                    77
Mag Patrimoine RES                    391
Mag Patrimoine RES 4                 1663
Mag Patrimoine RES 8                 1829
Mag Patrimoine RES Fol                223
Mag Patrimoine RES Plano               19
Mag Patrimoine fol.                  1346
Mag Patrimoine °4                    5922
Mag Patrimoine °8                   13696
Mag Pôle adultes BD                   961
Mag Pôle adultes fol                 1014
Mag Pôle adultes °4                   613
Mag Pôle adultes °8                 24837
Magasin collectivités               33559
Magasin disco 2e étag

In [58]:
anomalies_loc_incoherente_med = items[(items['site propriétaire'] == 'MED') & (items['location'].str[0:3] != 'MED')&(~items['location'].isna())]

In [59]:
anomalies_loc_incoherente_med.groupby(["site propriétaire","localisation"]).size()

site propriétaire  localisation    
MED                Musée La Piscine    7
                   Quarantaine         7
                   Zèbre               5
dtype: int64

In [60]:
anomalies_loc_incoherente_bus = items[(items["site propriétaire"] == "BUS") & (items["location"] != "BUS1A")]

In [61]:
anomalies_loc_incoherente_bus.groupby(["site propriétaire","localisation"]).size()

site propriétaire  localisation         
BUS                1er étage                2
                   2e étage -  Jeunesse     1
                   Magasin collectivités    1
                   Rez-de-chaussée          6
dtype: int64

In [62]:
anomalies_loc_incoherente_med[colonnes_a_exporter].to_excel("../data/liste_anomalies_localisation_incoherente_med.xlsx",index=False)

In [63]:
anomalies_loc_incoherente_med[colonnes_a_exporter].to_excel("../data/liste_anomalies_localisation_incoherente_bus.xlsx",index=False)

## Anomalies sur les données manquantes
### Code-barre manquant
Voici une description de la méthode employée pour la liste des anomalies de code-barre manquant : 
* J'ai sélectionné tous les documents dont le code-barre est manquant via la fonction `.isna()`
* J'ai ensuite exclu de la requête les documents *En commande* et *En traitement* 
* Ne restent dans les anomalies de code-barre que les documents sans code-barre avec un statut problématique.

In [64]:
codebarre_ko = items[items["code-barre"].isna()]

In [65]:
codebarre_ko.groupby(["statut","localisation"]).size()

statut             localisation          
1 - En commande    1er étage                 260
                   Rez-de-chaussée            39
2 - En traitement  1er étage                   1
                   2e étage -  Jeunesse        8
                   3e étage                    1
                   Magasin collectivités       2
                   Magasin disco 2e étage      1
3 - Empruntable    1er étage                   3
                   2e étage -  Jeunesse        2
                   3e étage                    1
                   Magasin collectivités       1
6 -  En retrait    1er étage                   1
dtype: int64

In [66]:
anomalies_docs_codebarre_ko = codebarre_ko[~codebarre_ko["notforloan"].isin([-2,-1])]

In [67]:
anomalies_docs_codebarre_ko.groupby(["statut"]).size()

statut
3 - Empruntable      12
5 - En réparation     4
6 -  En retrait       2
dtype: int64

In [68]:
anomalies_docs_codebarre_ko[colonnes_a_exporter].to_excel('../data/liste_anomalies_exemplaires_codebarre_ko.xlsx',index=False)

### Site propriétaire manquant

In [69]:
site_proprietaire_ko = items[items["site propriétaire"].isna()]
site_proprietaire_ko

,itemnumber,biblionumber,biblioitemnumber,code-barre,date d'acquisition,booksellerid,site propriétaire,prix,coût de remplacement,replacementpricedate,...,new_status,exclude_from_local_holds_priority,itemtype,statut,abimé,perdu,retiré de la circulation,collection,localisation,type de support


### Localisation manquante 

In [70]:
localisation_ko = items[items["localisation"].isna()]
localisation_ko

,itemnumber,biblionumber,biblioitemnumber,code-barre,date d'acquisition,booksellerid,site propriétaire,prix,coût de remplacement,replacementpricedate,...,new_status,exclude_from_local_holds_priority,itemtype,statut,abimé,perdu,retiré de la circulation,collection,localisation,type de support
9748,452997,14495,14495,C3500017279,2022-11-25,None,BUS,NaN,9.20,2022-11-25,...,None,NaN,LI,2 - En traitement,Non,Non,Non,JAB - Albums petits enfants,NaN,Livre
15148,338456,22404,22404,C0005930319,2016-03-23,None,MED,13.57,13.57,2016-03-23,...,None,NaN,LI,4 - Réservé à la consultation sur place,Non,Non,Non,PPE - Patrimoine écrit - fonds général,NaN,Livre
15327,452909,22764,22764,C3500017297,2022-11-24,None,BUS,NaN,13.00,2022-11-24,...,None,NaN,LI,2 - En traitement,Non,Non,Non,JAB - Albums petits enfants,NaN,Livre
15958,28306,23762,23762,C0000012659,2005-03-23,None,MED,10.00,10.00,None,...,None,NaN,K7,3 - Empruntable,Non,Non,Non,PEN - FLRS,NaN,Cassette audio
43112,76166,67056,67056,C0002142581,2005-03-24,None,MED,4.29,4.29,None,...,None,NaN,LI,6 - En retrait,Non,Non,Non,JAB - Contes pour enfants,NaN,Livre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287795,451682,350695,350695,None,2022-11-02,None,MED,NaN,NaN,2022-11-02,...,None,NaN,VD,3 - Empruntable,Non,Non,Non,NaN,NaN,DVD
288145,452243,351079,351079,C3500016919,2022-11-12,None,BUS,NaN,7.95,2022-11-12,...,None,NaN,LI,3 - Empruntable,Non,Non,Non,JDT - 700 Activités manuelles / Arts,NaN,Livre
288160,452248,351094,351094,C3500016821,2022-11-12,None,BUS,NaN,12.90,2022-11-12,...,None,NaN,LI,3 - Empruntable,Non,Non,Non,JBD - E BD,NaN,Livre
288529,452937,351539,351539,C3500017269,2022-11-24,None,BUS,NaN,6.80,2022-11-24,...,None,NaN,LI,2 - En traitement,Non,Non,Non,JAB - Albums petits enfants,NaN,Livre


In [71]:
localisation_ko.groupby(["statut","notforloan"]).size()

statut                                   notforloan
1 - En commande                          -1              6
2 - En traitement                        -2             96
3 - Empruntable                           0            485
4 - Réservé à la consultation sur place   2            102
5 - En réparation                        -4             13
6 -  En retrait                          -3             10
8 - Sorti des collections                 4             34
dtype: int64

In [72]:
anomalies_localisation_ko = localisation_ko[~localisation_ko["notforloan"].isin([-1,-2,4])]
anomalies_localisation_ko

,itemnumber,biblionumber,biblioitemnumber,code-barre,date d'acquisition,booksellerid,site propriétaire,prix,coût de remplacement,replacementpricedate,...,new_status,exclude_from_local_holds_priority,itemtype,statut,abimé,perdu,retiré de la circulation,collection,localisation,type de support
15148,338456,22404,22404,C0005930319,2016-03-23,None,MED,13.57,13.57,2016-03-23,...,None,NaN,LI,4 - Réservé à la consultation sur place,Non,Non,Non,PPE - Patrimoine écrit - fonds général,NaN,Livre
15958,28306,23762,23762,C0000012659,2005-03-23,None,MED,10.00,10.00,None,...,None,NaN,K7,3 - Empruntable,Non,Non,Non,PEN - FLRS,NaN,Cassette audio
43112,76166,67056,67056,C0002142581,2005-03-24,None,MED,4.29,4.29,None,...,None,NaN,LI,6 - En retrait,Non,Non,Non,JAB - Contes pour enfants,NaN,Livre
48662,85386,75351,75351,C0005555277,2012-09-26,None,BUS,11.50,11.50,None,...,None,NaN,LI,5 - En réparation,Abimé,Non,Non,JAB - Contes petits enfants,NaN,Livre
55287,432377,84901,84901,None,2021-09-18,None,MED,NaN,NaN,2021-09-18,...,None,NaN,PA,6 - En retrait,Non,Non,Non,NaN,NaN,Partition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287628,450234,350547,350547,C3500016800,2022-10-07,None,BUS,NaN,NaN,2022-10-07,...,None,NaN,VD,3 - Empruntable,Non,Non,Non,JCI - Films pour enfants - à partir de 6 ans,NaN,DVD
287770,450645,350667,350667,C3500016875,2022-10-12,None,BUS,NaN,NaN,2022-10-12,...,None,NaN,VD,3 - Empruntable,Non,Non,Non,JCI - Dessins animés - Enfants,NaN,DVD
287795,451682,350695,350695,None,2022-11-02,None,MED,NaN,NaN,2022-11-02,...,None,NaN,VD,3 - Empruntable,Non,Non,Non,NaN,NaN,DVD
288145,452243,351079,351079,C3500016919,2022-11-12,None,BUS,NaN,7.95,2022-11-12,...,None,NaN,LI,3 - Empruntable,Non,Non,Non,JDT - 700 Activités manuelles / Arts,NaN,Livre


In [73]:
anomalies_localisation_ko[colonnes_a_exporter].to_excel("../data/liste_anomalies_exemplaires_localisation_ko.xlsx",index=False)

### Prix manquant

Voici une description de la méthode employée pour générer la liste des anomalies d'exemplaires sans prix :
* J'ai défini la variable `prix_ko` comme la liste des exemplaires sans code-barre renseigné
* J'ai ensuite exclu de cette liste les documents du magasin, ainsi que les périodiques (défini par la variable `anomalies_prix_ko_coll`
* Grâce à un `groupby` et `.dt.year`, j'ai généré une liste du total de documents sans prix par an.
* On constate que c'est **à partir de 2020 que les documents sans prix augmentent**
* J'ai donc redéfini ma variable `anomalies_prix_ko_coll` comme la liste des exemplaires acheté il y a moins de 3 ans

> * [ ] Il faudrait voir à filtrer et exclure de la liste les docs du Musée (il y en a que 2) et du Zèbre

In [74]:
prix_ko = items[items["prix"].isna()]
prix_ko["localisation"] = prix_ko["localisation"].astype(str)

/tmp/ipykernel_1967/3213706575.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prix_ko["localisation"] = prix_ko["localisation"].astype(str)


In [75]:
anomalies_prix_ko_coll = prix_ko[~prix_ko["localisation"].str.contains('Mag')]

In [76]:
anomalies_prix_ko_coll = anomalies_prix_ko_coll[~anomalies_prix_ko_coll.localisation.str.contains('Pé')]

In [77]:
anomalies_prix_ko_coll.groupby([anomalies_prix_ko_coll["date d'acquisition"].dt.year]).size()

date d'acquisition
2005    4374
2006       3
2007       2
2008     189
2009     284
2010     606
2011     136
2012       5
2013       4
2014      25
2015      19
2016     226
2017     434
2018     582
2019    1129
2020    1743
2021    3646
2022    4918
dtype: int64

In [78]:
anomalies_prix_ko_coll = anomalies_prix_ko_coll[anomalies_prix_ko_coll["date d'acquisition"].dt.year >= 2020]

In [79]:
anomalies_prix_ko_coll.groupby(["localisation","collection"]).size()

localisation  collection                                        
1er étage     AAP - Arts plastiques                                 233
              AAP - Bande dessinée                                   15
              AAP - Dessin animé et animation (documentaires)        11
              AAP - Documentaires sur le cinéma et la télévision    100
              AAP - Documentaires sur les jeux vidéo                 96
                                                                   ... 
nan           JRO - Premiers romans                                  17
              JRO - Romans policiers pour enfants                     2
              JRO - Romans pour enfants                              26
              PPE - Patrimoine écrit - fonds général                  2
              PPE - Périodiques patrimonaiux                          3
Length: 169, dtype: int64

In [80]:
anomalies_prix_ko_coll[colonnes_a_exporter].to_excel("../data/liste_anomalies_exemplaires_prix_ko_coll.xlsx",index=False)

### Collection manquante

In [81]:
collection_ko = items[items["collection"].isna()]

In [82]:
anomalies_collection_ko = collection_ko[~collection_ko["notforloan"].isin([-1,-2,3,4])]

In [83]:
anomalies_collection_ko.groupby(["notforloan","statut"]).size()

notforloan  statut                                 
-4          5 - En réparation                            13
-3          6 -  En retrait                              96
 0          3 - Empruntable                            2214
 2          4 - Réservé à la consultation sur place    6448
 5          7 - En reliure                                1
dtype: int64

In [84]:
anomalies_collection_ko[colonnes_a_exporter].to_excel("../data/liste_anomalies_exemplaires_collection_ko.xlsx",index=False)

### Côte manquante

In [85]:
cote_ko = items[items["cote"].isna()]
cote_ko.groupby(["statut","notforloan"]).size()

statut                                   notforloan
1 - En commande                          -1            301
2 - En traitement                        -2             76
3 - Empruntable                           0             82
4 - Réservé à la consultation sur place   2             73
5 - En réparation                        -4              4
6 -  En retrait                          -3              5
8 - Sorti des collections                 4              3
9 - Consultable au musée                  3              1
dtype: int64

In [86]:
anomalies_cote_ko = cote_ko[~cote_ko["notforloan"].isin([-1,-2,4])]
anomalies_cote_ko.groupby("localisation").size()

localisation
1er étage                           31
2e étage -  Jeunesse                 9
2e étage - Salle d'étude             7
3e étage                             5
Mag Bibliothèque professionnelle     1
Mag Patrimoine                       1
Mag Patrimoine BR 8                  3
Mag Patrimoine °4                    3
Mag Patrimoine °8                    7
Mag Pôle adultes °8                 24
Magasin disco 2e étage               1
Magasin disco 3e étage              40
Musée La Piscine                    16
Rez-de-chaussée                      2
Zèbre                                1
dtype: int64

In [87]:
anomalies_cote_ko[colonnes_a_exporter].to_excel('../data/liste_anomalies__exemplaires_docs_cote_ko.xlsx',index=False)

### Type de support manquant

In [88]:
anomalies_type_support_ko = items[items["type de support"].isna()]
anomalies_type_support_ko.groupby(["statut"]).size()

statut
3 - Empruntable                            122
4 - Réservé à la consultation sur place      5
dtype: int64

In [89]:
anomalies_type_support_ko[colonnes_a_exporter].to_excel("../data/liste_anomalies_exemplaires_type_support_ko.xlsx",index=False)

## Anomalies sur les statuts

In [90]:
items.groupby(["notforloan","statut"]).size()

notforloan  statut                                 
-4          5 - En réparation                             909
-3          6 -  En retrait                              1970
-2          2 - En traitement                            1614
-1          1 - En commande                               305
 0          3 - Empruntable                            218794
 2          4 - Réservé à la consultation sur place     55479
 3          9 - Consultable au musée                      595
 4          8 - Sorti des collections                    9504
 5          7 - En reliure                                106
dtype: int64

### Documents abimés

Voici une description de la méthode employée pour générer la liste des anomalies d'exemplaires abimés : 
* J'ai d'abord affiché une liste des valeurs pour la colonne `abimé` (cela servira pour les requêtes)
* J'ai ensuite défini la variable `docs_abimés` comme la liste des documents marqués en `Abimé`
* J'ai ensuite défini la variable `anomalies_docs_abimes`comme la liste des docs abimés ayant été marqué en abimé depuis plus d'un an (grâce à la variable `deadline_1_an`)
* J'ai ensuit utilisé la fonction `groupby` afin d'afficher la liste des docs abimés par statut
* J'ai pu filtrer et générer deux listes différentes grâce àa la fonction `.localisation.str.contains` : 
 - la première comprends les anomalies localisées ailleurs que dans les magasins `anomalies_docs_abimes_coll`
 - La seconde liste concerne les anomalies localisées dans les différents magasins `anomalies_docs_abimes_mag` 
* **On a comme résultat une liste des documents marqués comme abimés depuis plus d'un an et qui ne sont ni sortis des collections ni des documents localisés au magasin**

In [91]:
items["abimé"].value_counts()

Non               285834
Abimé               3384
Prétendu rendu        58
Name: abimé, dtype: int64

In [92]:
docs_abimes = items[items["abimé"]=="Abimé"]

In [93]:
anomalies_docs_abimes = docs_abimes[docs_abimes["abimé le"]<deadline_1_an]

In [94]:
anomalies_docs_abimes.groupby("statut").size()

statut
2 - En traitement              1
3 - Empruntable              248
5 - En réparation             63
6 -  En retrait                8
8 - Sorti des collections    127
dtype: int64

In [95]:
anomalies_docs_abimes = anomalies_docs_abimes[~anomalies_docs_abimes["notforloan"].isin([4])]

In [96]:
anomalies_docs_abimes["localisation"] = anomalies_docs_abimes["localisation"].astype(str)

In [97]:
anomalies_docs_abimes_coll = anomalies_docs_abimes[~anomalies_docs_abimes.localisation.str.contains('Mag')]

In [98]:
anomalies_docs_abimes_mag = anomalies_docs_abimes[anomalies_docs_abimes.localisation.str.contains('Magasin')]

In [99]:
anomalies_docs_abimes.groupby(["statut","localisation"]).size()

statut             localisation          
2 - En traitement  1er étage                   1
3 - Empruntable    1er étage                  50
                   2e étage -  Jeunesse       61
                   3e étage                    9
                   Mag Pôle adultes °4         1
                   Mag Pôle adultes °8         1
                   Magasin collectivités     105
                   Magasin disco 3e étage      1
                   Magasin jeunesse            2
                   Rez-de-chaussée             9
                   Zèbre                       9
5 - En réparation  1er étage                   5
                   2e étage -  Jeunesse       32
                   3e étage                    2
                   Mag Pôle adultes BD         1
                   Magasin collectivités       7
                   Rez-de-chaussée             1
                   Zèbre                      14
                   nan                         1
6 -  En retrait    1er étag

In [100]:
anomalies_docs_abimes_mag.groupby(["statut","localisation"]).size()

statut             localisation          
3 - Empruntable    Magasin collectivités     105
                   Magasin disco 3e étage      1
                   Magasin jeunesse            2
5 - En réparation  Magasin collectivités       7
dtype: int64

On obtient ici la liste des exemplaires passés en abimé depuis plus d'un an (sont exclus de cette liste les exemplaires sortis des collections).

In [101]:
anomalies_docs_abimes_coll[colonnes_a_exporter].to_excel('../data/liste_anomalies_exemplaires_abimes_coll.xlsx',index=False)

In [102]:
anomalies_docs_abimes_coll[colonnes_a_exporter].to_excel('../data/liste_anomalies_exemplaires_abimes_mag.xlsx',index=False)

### Documents perdus
Les documents perdus font déjà l'objet d'un traitement. Toutes les 1, 3 et 5 semaines, François a mis en place un script qui génère une liste des documents perdus.
> Réfléchir peut-être à une requête du style : docs marqué en perdu depuis plus d'un an et qui ont été bipé il y a moins de 5 semaine (grâce à `timestamp|`)

### Documents en retrait

In [103]:
docs_en_retrait = items[items['notforloan'].isin([-3])]
docs_en_retrait_codebarre_ko = docs_en_retrait[docs_en_retrait["code-barre"].isna()]
docs_en_retrait_codebarre_ko

,itemnumber,biblionumber,biblioitemnumber,code-barre,date d'acquisition,booksellerid,site propriétaire,prix,coût de remplacement,replacementpricedate,...,new_status,exclude_from_local_holds_priority,itemtype,statut,abimé,perdu,retiré de la circulation,collection,localisation,type de support
55287,432377,84901,84901,None,2021-09-18,None,MED,NaN,NaN,2021-09-18,...,None,NaN,PA,6 - En retrait,Non,Non,Non,NaN,NaN,Partition
279686,436666,343364,343364,None,2021-12-14,None,MED,NaN,NaN,2021-12-14,...,None,NaN,LS,6 - En retrait,Non,Non,Non,ALT - Livres audio,1er étage,Livre audio


In [104]:
anomalies_docs_en_retrait = docs_en_retrait[docs_en_retrait["date dernier changement"] < deadline_1_an]
anomalies_docs_en_retrait.groupby(['localisation',"collection"]).size()

localisation     collection                                        
1er étage        AAP - Arts plastiques                                  2
                 AAP - Dessin animé et animation                        3
                 AAP - Documentaires sur le cinéma et la télévision     4
                 AAP - Films                                            3
                 AAP - PL - Cuisine et bricolage                        3
                                                                       ..
Rez-de-chaussée  PRR - FLRS de prêt                                     1
                 PRR - Région                                          32
Zèbre            ALT - Romans policiers                                 1
                 JAB - Contes pour enfants                              1
                 JCI - Dessins animés - Enfants                         1
Length: 120, dtype: int64

In [105]:
anomalies_docs_en_retrait[colonnes_a_exporter].to_excel('../data/liste_anomalies_exemplaires_docs_en_retrait.xlsx',index=False)

### Documents en commande

> J'ai décidé arbitrairement de considérer comme des anomalies les documents en commande depuis 6 mois ou plus. À voir avec François

In [106]:
docs_en_commande = items[items['notforloan'].isin([-1])]

In [107]:
anomalies_docs_en_commande = docs_en_commande[docs_en_commande['date dernier changement'] < deadline_6_mois]

In [108]:
anomalies_docs_en_commande.groupby(items['location']).size()

location
MED0C    36
MED1A    24
dtype: int64

In [109]:
anomalies_docs_en_commande[colonnes_a_exporter].to_excel('../data/liste_anomalies_exemplaires_docs_en_commande.xlsx',index=False)

### Documents en réparation

In [110]:
docs_en_reparation = items[items['notforloan'].isin([-4])]
docs_en_reparation.groupby("localisation").size()

localisation
1er étage                  40
2e étage -  Jeunesse      242
3e étage                   36
Mag Patrimoine °8           3
Mag Pôle adultes BD       127
Mag Pôle adultes fol        6
Mag Pôle adultes °8        35
Magasin collectivités     281
Magasin disco 3e étage      1
Magasin jeunesse            2
Rez-de-chaussée            80
Zèbre                      43
dtype: int64

In [111]:
anomalies_docs_en_reparation = docs_en_reparation[docs_en_reparation["date dernier changement"] < deadline_1_an]
anomalies_docs_en_reparation.groupby("localisation").size()

localisation
1er étage                 20
2e étage -  Jeunesse      53
3e étage                   8
Mag Patrimoine °8          2
Mag Pôle adultes BD        2
Mag Pôle adultes fol       2
Mag Pôle adultes °8        2
Magasin collectivités    113
Rez-de-chaussée           10
Zèbre                     19
dtype: int64

In [112]:
# Réfléchir à une deadline pour les Collectivités. Agnès me disait que les filles réparent l'hiver.

In [113]:
anomalies_docs_en_reparation[colonnes_a_exporter].to_excel('../data/liste_anomalies_exemplaires_docs_en_reparation.xlsx',index=False)

### Documents en reliure

In [114]:
docs_en_reliure = items[items["notforloan"]== 5]
docs_en_reliure.groupby("localisation").size()

localisation
1er étage                1
2e étage -  Jeunesse     1
Mag Patrimoine RES 4     1
Mag Pôle adultes fol    93
Rez-de-chaussée         10
dtype: int64

In [115]:
docs_en_reliure[colonnes_a_exporter].to_excel('../data/liste_anomalies_exemplaires_docs_en_reliure.xlsx',index=False)